In [8]:
import json
import requests
import os

# os.environ['NO_PROXY'] = '127.0.0.1'
url = "http://127.0.0.1:5000"

## Get info


In [9]:
response = requests.post(url + "/routeGetDatasetsInfo", {})
# dataset = 'ascertain'
dataset = 'case_categorical'
json.loads(response.content)

{'loadedDatasetsIds': ['ascertain', 'case_categorical'],
 'localDatasetsIds': ['ascertain',
  'drivers_workload',
  'drivers_stress',
  'wesad_dimensional_2',
  'wesad_dimensional_3',
  'wesad_categorical_panas',
  'wesad_categorical_stai',
  'case_dimensional',
  'case_categorical']}

## Load local dataset

In [3]:
response = requests.post(url + "/loadLocalDataset",data = {'datasetId':dataset})
json.loads(response.content)

{'state': 'success'}

## Remove dataset

In [17]:
response = requests.post(url + "/removeDataset",data = {'datasetId':dataset})
json.loads(response.content)

{'state': 'error'}

## Get Dataset info

In [10]:
response = requests.post(url + "/getDatasetInfo",data = {'datasetId':dataset})

info = json.loads(response.content)
# info

## Get Time Series

In [5]:
# response = requests.post(url + "/getMTSeries",data = {'datasetId':dataset, 'begin': 0, 'end': 5, 'ids': json.dumps(['s4', 's2', 's5'])})
# json.loads(response.content)

## Get projection


In [11]:
alphas = {label: 1 for label in info["seriesLabels"]}
D_k = {}
coords = {}

In [12]:
response = requests.post(url + "/getDatasetProjection",data = {'datasetId':dataset, 'alphas': json.dumps(alphas), 'begin': 0, 'end':2000, 
                                                               'D_k': json.dumps(D_k), 'oldCoords': json.dumps(coords), 'distance': 2,
                                                               'projection':0, 'projectionParameter' : 5
                                                              })
data = json.loads(response.content)
coords = data["coords"]
D_k = data["D_k"]
coords

{'1': [-0.34655396623684254, -0.31354615043304923],
 '10': [0.030157836319931297, 0.5206156284950145],
 '11': [-0.7484552651029182, -0.45845270674805993],
 '12': [0.7063725522191042, -0.46845505650691244],
 '13': [-0.6343976133938124, 0.3820419609084251],
 '14': [0.8362588749819571, 0.5712173289482743],
 '15': [-0.6425594679633474, -0.751238187173185],
 '16': [0.8906119164505464, -0.28236241486152025],
 '17': [-0.8734530466904246, 0.08270747650758281],
 '18': [0.12007087636637137, 1.0],
 '19': [0.06917288040819214, -0.7037009039089623],
 '2': [-0.9184963749478082, -0.2119702516820896],
 '20': [0.3443699086971692, -0.937804076377504],
 '21': [0.49515345020585144, 0.5960588095350137],
 '22': [-0.23629958644600602, 0.650677439857392],
 '23': [0.35081034772221154, -0.37556514855773027],
 '24': [0.8384303788138752, 0.2613785842281455],
 '25': [0.5608387475333534, -0.7095065600030221],
 '26': [-0.45073678032875564, 0.018173243907628756],
 '27': [-0.5442450552720236, 0.7012157164741983],
 '28

## Clustering

In [38]:
response = requests.post(url + "/doClustering",data = {'datasetId':dataset, 'coords': json.dumps(coords), 'k': 2})
clusters = json.loads(response.content)
clusters

{'0': ['1', '2', '4', '5'], '1': ['3', '6']}

## Rankings

In [39]:
response = requests.post(url + "/getFishersDiscriminantRanking",data = {'datasetId':dataset, 'D_k': json.dumps(D_k), 'blueCluster': json.dumps(clusters['0']), 'redCluster': json.dumps(clusters['1']) })
j_s = json.loads(response.content)
j_s

{'Arousal': 0.051812307045928346, 'Valence': 3.415525686489257}

## Temporal summary

In [32]:
response = requests.post(url + "/getTemporalSummary",data = {'datasetId':dataset})
# json.loads(response.content)["mean"]


## Downsample

In [36]:
response = requests.post(url + "/downsampleData",data = {'datasetId':dataset, 'rule': 'T'})
json.loads(response.content)

{'state': 'success'}

## Reset dataset

In [68]:
response = requests.post(url + "/resetDataset",data = {'datasetId':dataset})
json.loads(response.content)

{'state': 'success'}